In [ ]:
import fsspec

fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_ssl=False) # For now SSL false is solving my cert issues **shrug**
fs_write = fsspec.filesystem('')

In [ ]:
hrr_subhourly_member_files = fs_read.glob('s3://noaa-hrrr-bdp-pds/hrrr.20230720/conus/hrrr.t23z.wrfsubhf*.grib2')

files = sorted(['s3://'+f for f in hrr_subhourly_member_files])
print(f'Read {len(files)} HRRR files')

In [ ]:
import ujson
from gribberish.kerchunk import scan_gribberish

so = {"anon": True, "use_ssl": False}
json_dir = 'hrrr/'

def make_json_name(file_url, message_number): #create a unique name for each reference file
    date = file_url.split('/')[3].split('.')[1]
    name = file_url.split('/')[5].split('.')[0:3]
    return f'{json_dir}{date}_{name[0]}_{name[1]}_{name[2]}_message{message_number}.json'

def gen_json(file_url):
    out = scan_gribberish(file_url, storage_options=so, only_variables=['prate', 'ugrd', 'vgrd', 'tmp'], perserve_dims=['hag'], filter_by_attrs={'statistical_process': ''}) 
    for i, message in enumerate(out):
        out_file_name = make_json_name(file_url, i)  # get name
        with fs_write.open(out_file_name, "w") as f: 
            f.write(ujson.dumps(message)) # write to file

In [ ]:
from dask.distributed import Client, progress

client = Client()

In [ ]:
futures = client.map(gen_json, files[1:], retries=1)
progress(futures)

In [ ]:
client.shutdown()

In [ ]:
from kerchunk.combine import MultiZarrToZarr

reference_jsons = sorted(fs_write.ls(json_dir)) #get list of file names

print(f'Found {len(reference_jsons)} reference files')

# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(reference_jsons,
                      concat_dims = ['time'],
                      identical_dims=['x', 'y', 'latitude', 'longitude', 'hag'])

d = mzz.translate()

with open(f'{json_dir}/hrrr_kerchunk.json', 'w') as f:
    f.write(ujson.dumps(d))

In [ ]:
import xarray as xr

# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo=f'./hrrr/hrrr_kerchunk.json', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), chunks={'time': 1})
ds

In [ ]:
import pyproj
to_xy = pyproj.Transformer.from_crs('epsg:4326', ds.apcp.crs, always_xy=True).transform

In [ ]:
lat, lng = 41.42717976016072, -71.462122760827
x_sel, y_sel = to_xy(lng, lat)

In [ ]:
from matplotlib import pyplot as plt

f, ax = plt.subplots(figsize=(10, 4))

selected_ds = ds.interp(x=x_sel, y=y_sel, method='linear')

selected_prate = (selected_ds.prate * 3600) / 25.4 # convert from kg/m^2/s to in/hr
selected_apcp = selected_ds.apcp / 25.4 # convert from kg/m^2/s to in/hr

#selected_prate.plot.line(ax=ax)
selected_apcp.plot.line(ax=ax)